# Computes the correlation coefficient between C8 and C16

cannot be run without hdf5 file contain all C8 and C16 data

In [19]:
from analysis import *
import h5py
from astropy.convolution import convolve, Box1DKernel
import sys

#if __name__=="__main__":

for channel in range(5,85):

    campaign = 8
    mod,submod = get_submod(channel)
    submod-=1 # submods are 0-3 in the hdf5 file

    #hdf5_file = "/Users/rachelbuttry/K2/K2C%s_target_pixels.hdf5"%campaign
    hdf5_file = "/home/jackeline/Research/k2_pipeline/k2_LongTermSystematics/data/pixelmaps/k2pixelmap.hdf5"

    #hdf5_file = "/home/rachel/Research/K2/K2C%s_target_pixels.hdf5"%campaign
    # there are 23117 objects in the campaign 8 hdf5 file
    # there are 3853 cadence points in c8
    cadences = 3853
    time = np.arange(cadences)/48.0
    
    try:
        with h5py.File(hdf5_file, 'r') as f:
            channel_epics = np.array(list(f['%s/%s/%s'%(campaign, mod,submod)].keys()))# look at objects in given channel).astype(int)

        # need the kepler magnitudes
        #all_targets = pd.read_csv("./K2_official_targets/K2Campaign%stargets.csv"%campaign, skipinitialspace=True)
        all_targets = pd.read_csv("/home/jackeline/Research/k2_pipeline/K2_official_targets/K2Campaign%stargets.csv"%campaign, skipinitialspace=True)


        # saving lcs to take the median
        arr = []

        with h5py.File(hdf5_file, 'r') as f:
            channel_epics = np.array(list(f['%s/%s/%s'%(campaign, mod, submod)].keys()))# look at objects in given channel).astype(int)

            rel_epics = channel_epics[np.isin(channel_epics, all_targets['EPIC ID'][np.logical_and(all_targets['magnitude'] > 13, all_targets['magnitude'] < 20)])]
            #print(len(rel_epics))

            # loop thru the objects
            for epic in rel_epics:
                d = np.array(f['%s/%s/%s/%s'%(campaign, mod, submod, epic)]['data'])
                lc_hdf5 = np.nansum(np.nansum(d, axis=1), axis=1)

                # handle spurious cadences
                lc = lk.LightCurve(time, flux=lc_hdf5)
                _, spurious_cad = lc.flatten().remove_outliers(return_mask=True) # remove spurious cadences
                lc_raw = lc.flux

                # interpolate
                cadno = np.arange(len(lc_raw)) # get an array to serve as our time/cadence measurement
                interped_vals = np.interp(cadno[spurious_cad], cadno[~spurious_cad], lc_raw[~spurious_cad])
                # replace spurious cadence values with the interpolated values
                lc_raw[spurious_cad] = interped_vals
                norm = np.std(lc_raw)
                lc_raw -= np.mean(lc_raw)
                lc_raw = lc_raw/norm
                #lc_raw = lc_raw/np.max(np.abs(lc_raw))

                smooth = convolve(lc_raw, Box1DKernel(350), boundary='extend')

                arr.append(lc_raw)

        cmap = plt.get_cmap('inferno')
        mag_colors = np.zeros((len(cmap.colors),4))
        mag_colors[:,3] = 0.7 #this is the alpha parameter
        mag_colors[:,:3] = cmap.colors




        #fig = plt.figure(figsize=(8,5))

        p= 50
        flux = np.percentile(np.atleast_2d(np.array(arr)),p, axis=0)
        flux_smooth = convolve(flux, Box1DKernel(250), boundary='extend')
        #plt.plot(time, flux, linewidth=4, alpha=0.8, color="#fee6ce")
        #plt.plot(time, flux_smooth-flux_smooth.mean(), linewidth=10, color="#e6550d" )
        lcList[channel]  =  flux_smooth

    except:
        pass

In [12]:
lcList[channel]  =  flux_smooth

In [21]:
import json

In [25]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open('c8_fig4_lcs.p', 'wb') as fp:
    pickle.dump(lcList, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [26]:
lc16List = {}

In [27]:
for channel in range(2,85):

    campaign = 16
    mod,submod = get_submod(channel)
    submod-=1 # submods are 0-3 in the hdf5 file

    #hdf5_file = "/Users/rachelbuttry/K2/K2C%s_target_pixels.hdf5"%campaign
    hdf5_file = "/home/jackeline/Dropbox/Kepler/K2PixelMap_c16.hdf5"

    #hdf5_file = "/home/rachel/Research/K2/K2C%s_target_pixels.hdf5"%campaign
    # there are 3894 cadence points in c16
    cadences = 3894
    time = np.arange(cadences)/48.0
    try:
        
        with h5py.File(hdf5_file, 'r') as f:
            channel_epics = np.array(list(f['%s/%s/%s'%(campaign, mod,submod)].keys()))# look at objects in given channel).astype(int)

        # need the kepler magnitudes
        all_targets = pd.read_csv("/home/jackeline/Research/k2_pipeline/K2_official_targets/K2Campaign%stargets.csv"%campaign, skipinitialspace=True)

        # saving lcs to take the median
        arr = []

        with h5py.File(hdf5_file, 'r') as f:
            channel_epics = np.array(list(f['%s/%s/%s'%(campaign, mod, submod)].keys()))# look at objects in given channel).astype(int)

            rel_epics = channel_epics[np.isin(channel_epics, all_targets['EPIC ID'][np.logical_and(all_targets['magnitude'] > 13, all_targets['magnitude'] < 20)])]
            #print(len(rel_epics))

            # loop thru the objects
            for epic in rel_epics:
                d = np.array(f['%s/%s/%s/%s'%(campaign, mod, submod, epic)]['data'])
                lc_hdf5 = np.nansum(np.nansum(d, axis=1), axis=1)

                # handle spurious cadences
                lc = lk.LightCurve(time, flux=lc_hdf5)
                _, spurious_cad = lc.flatten().remove_outliers(return_mask=True) # remove spurious cadences
                lc_raw = lc.flux

                # interpolate
                cadno = np.arange(len(lc_raw)) # get an array to serve as our time/cadence measurement
                interped_vals = np.interp(cadno[spurious_cad], cadno[~spurious_cad], lc_raw[~spurious_cad])
                # replace spurious cadence values with the interpolated values
                lc_raw[spurious_cad] = interped_vals
                norm = np.std(lc_raw)
                lc_raw -= np.mean(lc_raw)
                lc_raw = lc_raw/norm

                smooth = convolve(lc_raw, Box1DKernel(350), boundary='extend')

                arr.append(lc_raw)

        #flux = np.nanmedian(np.atleast_2d(np.array(arr)), axis=0)
        #flux_smooth = convolve(flux, Box1DKernel(250), boundary='extend')

        #fig = plt.figure(figsize=(8,5))
        #plt.plot(time, flux[::-1], linewidth=4, alpha=0.8, color="#9ebcda")
        #plt.plot(time, flux_smooth[::-1], linewidth=10, color="#8856a7")

        #uncomment everything above to revert back 



        #fig = plt.figure(figsize=(8,5))

        p = 50
        flux = np.percentile(np.atleast_2d(np.array(arr)),p, axis=0)
        flux_smooth = convolve(flux, Box1DKernel(250), boundary='extend')
        #plt.plot(time, flux[::-1], linewidth=4, alpha=0.8, color="#9ebcda")
        #plt.plot(time, (flux_smooth-flux_smooth.mean())[::-1], linewidth=10,  color="#8856a7")
        lc16List[channel] = flux_smooth
    except:
        print("no data for channel",channel)
        pass

    

no data for channel 5
no data for channel 6
no data for channel 7
no data for channel 8
no data for channel 9
no data for channel 10
no data for channel 11
no data for channel 12
no data for channel 17
no data for channel 18
no data for channel 19
no data for channel 20


In [29]:
#lc16List

In [30]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

with open('c16_fig4_lcs.p', 'wb') as fp:
    pickle.dump(lc16List, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [41]:
import scipy.stats as stats
chList = []
rList = []



for key in lc16List.keys():
    #print(len(lcList[key]), len(lc16List[key]))
    lc8 = lcList[key]
    lc16 = lc16List[key][:len(lcList[key])][::-1]
    r, p = stats.pearsonr(lc8 - lc8.mean(), lc16-lc16.mean())
    #f, ax = plt.subplots(1,1)
    #ax.plot(lc8)
    #ax.plot(lc16)





2 0.9101559407342444 0.0
3 0.7578638194956553 0.0
4 0.7365823124278774 0.0
13 0.8291466110384835 0.0
14 0.7629735962387325 0.0
15 0.2678515416122086 2.65254269774184e-64
16 0.847434840384153 0.0
21 0.8270160403483373 0.0
22 0.9448217913757206 0.0
23 0.8028211663612919 0.0
24 0.7923335565250504 0.0
25 -0.1277767012548306 1.6996040504085149e-15
26 0.696593156902126 0.0
27 0.20610220052954076 3.1215841812791796e-38
28 0.657185826039909 0.0
29 0.7773671797367858 0.0
30 0.5506510541217602 1.789844640043413e-304
31 0.5203058311238016 2.4983710760755033e-266
32 0.44228067232088586 2.7336591444292944e-184
33 0.8664449372610151 0.0
34 0.1519651403958161 2.4253227387899374e-21
35 0.6702332524722161 0.0
36 -0.09001914667358177 2.177530349531829e-08
37 0.8979633999410805 0.0
38 0.5784311306246581 0.0
39 -0.14363229822190404 3.269881529523216e-19
40 0.42221054247679546 2.0065300449116835e-166
41 0.9176351705961507 0.0
42 0.8151313830614761 0.0
43 0.848231363000705 0.0
44 0.9671651823969095 0.0
45 -

In [48]:
db = pd.DataFrame(np.array(rList).reshape(1,71), columns = chList)

In [49]:
db.to_csv("CorrelationCoeff_16_8.csv", index = False)